# Natural Language Processing - lab 4 (Multiword expressions identification and extraction)

Bartosz Klimza

# Necessary imports

In [1]:
import pl_core_news_sm
import spacy
from spacy.tokenizer import Tokenizer
from nltk.util import ngrams
import os
import math


In [2]:
!python -m spacy download pl_core_news_sm


     |████████████████████████████████| 48.7 MB 850 kB/s eta 0:00:01
✔ Download and installation successful
You can now load the model via spacy.load('pl_core_news_sm')


# 1. Use SpaCy tokenizer API to tokenize the text from the law corpus:

In [3]:
nlp = pl_core_news_sm.load()

# Create a blank Tokenizer with just the Polish vocab
tokenizer = Tokenizer(nlp.vocab)


# 2. Compute bigram counts of downcased tokens. Given the sentence: "The quick brown fox jumps over the lazy dog.", the bigram counts are as follows:

* "the quick":1
* "quick brown":1
* "brown fox":1
* ...
* "dog.":1

In [18]:
data_dir = "./ustawy/"

bigrams_dict = dict()
for file in os.listdir(data_dir):
    bills = open(f"./{data_dir}{file}", "r").read()
    bills = bills.split()
    bills = " ".join(bills)
    bills = bills.lower()
    tokens = tokenizer(bills)
    bigrams = ngrams(tokens, 2)
    bigrams_list = []
    for b in bigrams:
        if str(b[0])[-1] in [".", ":", ";", "(", ")", "[", "]", ","]:
            bigrams_list.append(str(b[0])[:-1]+str(b[0])[-1])
        elif str(b[1])[-1] in [".", ":", ";", "(", ")", "[", "]", ","]:
            bigrams_list.append(str(b[1])[:-1]+str(b[1])[-1])
        else:
            bigrams_list.append(str(b[0]) + " " + str(b[1]))
    for i in bigrams_list:
        if i in bigrams_dict.keys():
            bigrams_dict[i] += 1
        else:
            bigrams_dict[i] = 1

bigrams_dictionary = {k: v for k, v in sorted(bigrams_dict.items(), key=lambda item: item[1], reverse=True)}
bigrams_dictionary   

{'art.': 126873,
 'ust.': 103543,
 'r.': 62958,
 'poz.': 45369,
 'mowa w': 28450,
 '2.': 21587,
 '1.': 21312,
 'brzmienie:': 20974,
 '1,': 15672,
 '2)': 14309,
 'których mowa': 13847,
 'o których': 13839,
 '3.': 13730,
 '1)': 13695,
 'brzmieniu:': 13216,
 'z dnia': 9513,
 'którym mowa': 9165,
 'o którym': 9155,
 '2,': 9042,
 '3)': 9006,
 '(dz.u.': 8930,
 'do spraw': 8680,
 'i nr': 8435,
 'dodaje się': 8190,
 '4.': 8156,
 'rozporządzenia,': 8047,
 'w drodze': 7092,
 'na podstawie': 6610,
 'b)': 6356,
 'a)': 6255,
 'stosuje się': 6030,
 '4)': 5792,
 'określi,': 5615,
 '1 pkt': 5591,
 'o której': 5512,
 'której mowa': 5502,
 '3,': 5439,
 '5.': 5246,
 'w przypadku': 5096,
 'od dnia': 5084,
 'w zakresie': 5071,
 'właściwy do': 4835,
 'zastępuje się': 4776,
 '"art.': 4709,
 '| |': 4687,
 'w ustawie': 4605,
 'minister właściwy': 4282,
 'oraz z': 4256,
 'ustawy,': 4183,
 'w życie': 4123,
 '5)': 4030,
 'określonych w': 4025,
 'w tym': 3893,
 'w terminie': 3838,
 '(dz.': 3823,
 'lit.': 3783,
 '4

# 3. Discard bigrams containing characters other than letters. Make sure that you discard the invalid entries after computing the bigram counts:

In [19]:
bigrams_dictionary_copy = bigrams_dictionary.copy()
for k in bigrams_dictionary.keys():
    new_key = k.replace(" ", "")
    if not new_key.isalpha():
       del  bigrams_dictionary_copy[k]

bigrams_dictionary = bigrams_dictionary_copy
bigrams_dictionary


{'mowa w': 28450,
 'których mowa': 13847,
 'o których': 13839,
 'z dnia': 9513,
 'którym mowa': 9165,
 'o którym': 9155,
 'do spraw': 8680,
 'i nr': 8435,
 'dodaje się': 8190,
 'w drodze': 7092,
 'na podstawie': 6610,
 'stosuje się': 6030,
 'o której': 5512,
 'której mowa': 5502,
 'w przypadku': 5096,
 'od dnia': 5084,
 'w zakresie': 5071,
 'właściwy do': 4835,
 'zastępuje się': 4776,
 'w ustawie': 4605,
 'minister właściwy': 4282,
 'oraz z': 4256,
 'w życie': 4123,
 'określonych w': 4025,
 'w tym': 3893,
 'w terminie': 3838,
 'w pkt': 3746,
 'a także': 3674,
 'ustawie z': 3660,
 'ustawy z': 3056,
 'w razie': 3028,
 'się wyrazami': 2941,
 'może być': 2636,
 'w celu': 2614,
 'zgodnie z': 2542,
 'skreśla się': 2499,
 'dni od': 2473,
 'w szczególności': 2445,
 'wejścia w': 2372,
 'na wniosek': 2367,
 'się w': 2332,
 'z zastrzeżeniem': 2318,
 'z tytułu': 2304,
 'co najmniej': 2280,
 'się wyrazy': 2173,
 'do dnia': 2162,
 'w sprawach': 2148,
 'a w': 2141,
 'określone w': 2124,
 'rzeczypospo

# 4. Use pointwise mutual information to compute the measure for all pairs of words:

In [20]:
unigrams_dict = dict()
for file in os.listdir(data_dir):
    bills = open(f"./{data_dir}{file}", "r").read()
    bills = bills.split()
    bills = " ".join(bills)
    bills = bills.lower()
    tokens = tokenizer(bills)
    unigrams = ngrams(tokens, 1)
    unigrams_list = []
    for b in unigrams:
        b = str(b).replace(".", "").replace(":", "").replace(";", "").replace("(", "").replace(")", "").replace("[", "").replace("]", "").replace(",", "")
        unigrams_list.append(str(b))
    for i in unigrams_list:
        if i in unigrams_dict.keys():
            unigrams_dict[i] += 1
        else:
            unigrams_dict[i] = 1

unigrams_dictionary = {k: v for k, v in sorted(unigrams_dict.items(), key=lambda item: item[1], reverse=True)}

In [21]:
unigrams_dictionary_copy = unigrams_dictionary.copy()
for k in unigrams_dictionary.keys():
    if not k.isalpha():
       del  unigrams_dictionary_copy[k]

unigrams_dictionary = unigrams_dictionary_copy

In [22]:
unigrams_sum = sum(unigrams_dictionary.values())
bigrams_sum = sum(bigrams_dictionary.values())

def calculate_pmi(w_1, w_2):
    p_w_1 = unigrams_dictionary[w_1] / unigrams_sum
    p_w_2 = unigrams_dictionary[w_2] / unigrams_sum
    p_w_1_w_2 = bigrams_dictionary[w_1 + " " + w_2] / bigrams_sum
    return math.log(p_w_1_w_2 / (p_w_1 * p_w_2))


In [23]:
pmi_dict = dict()
for b in bigrams_dictionary.keys():
    w1 = b.split()[0]
    w2 = b.split()[1]
    pmi_dict[b] = calculate_pmi(w1, w2)

pmi_dict
    

{'mowa w': 3.2653291075699493,
 'których mowa': 4.959732737521558,
 'o których': 4.150482123697541,
 'z dnia': 3.535204012760526,
 'którym mowa': 4.966679459526838,
 'o którym': 4.156915051966411,
 'do spraw': 4.330654871788954,
 'i nr': 2.4116446796137874,
 'dodaje się': 4.7255341107464295,
 'w drodze': 3.2646692488774782,
 'na podstawie': 4.619568482484866,
 'stosuje się': 4.652294888303835,
 'o której': 4.104952649724248,
 'której mowa': 4.911809482910507,
 'w przypadku': 3.0347619696094634,
 'od dnia': 4.504761094247153,
 'w zakresie': 3.0960787009772024,
 'właściwy do': 4.246504439525977,
 'zastępuje się': 4.744712014518809,
 'w ustawie': 3.114323166422441,
 'minister właściwy': 6.297397056527112,
 'oraz z': 2.114622553842081,
 'w życie': 3.214658084931618,
 'określonych w': 3.0524368974979454,
 'w tym': 2.7868181861719266,
 'w terminie': 3.050769088259124,
 'w pkt': 1.6526039250441287,
 'a także': 5.377036765435371,
 'ustawie z': 3.7772755856035234,
 'ustawy z': 2.725906364709983

# 5. Sort the word pairs according to that measure in the descending order and determine top 10 entries:

In [24]:
pmi_dict = {k: v for k, v in sorted(pmi_dict.items(), key=lambda item: item[1], reverse=True)[:10]}
pmi_dict


{'doktorów habilitowanych': 15.485157506710125,
 'pionową ścianę': 15.485157506710125,
 'techno logii': 15.485157506710125,
 'usprawnianie zaburzonych': 15.485157506710125,
 'przepro wadza': 15.485157506710125,
 'stępkę położono': 15.485157506710125,
 'wybuchła wojna': 15.485157506710125,
 'dało pożytecznego': 15.485157506710125,
 'poświęcenie objęło': 15.485157506710125,
 'błędem nautycznym': 15.485157506710125}

# 6. Filter bigrams with number of occurrences lower than 5. Determine top 10 entries for the remaining dataset (>=5 occurrences):

In [25]:
bigrams_dictionary_copy = bigrams_dictionary.copy()
for k in bigrams_dictionary.keys():
    if bigrams_dictionary[k] < 5:
       del  bigrams_dictionary_copy[k]

bigrams_dictionary = bigrams_dictionary_copy
bigrams_dictionary_asc = {k: v for k, v in sorted(bigrams_dictionary.items(), key=lambda item: item[1], reverse=False)}
bigrams_dictionary_asc

{'zakres ustawowych': 5,
 'finansowe uzyskane': 5,
 'czasie korzystania': 5,
 'korzystaniem przez': 5,
 'lub szkodę': 5,
 'mleczne przeznaczone': 5,
 'indywidualna reprezentatywna': 5,
 'reprezentatywna zawartość': 5,
 'współczynnik przydziału': 5,
 'i wprowadzona': 5,
 'bezpośrednich w': 5,
 'rezerwę krajowej': 5,
 'i wprowadzonego': 5,
 'referencyjnym oraz': 5,
 'ramach przyznanej': 5,
 'przyznanej indywidualnej': 5,
 'skupującym w': 5,
 'mleka od': 5,
 'mleka sprzedanego': 5,
 'obliczonej w': 5,
 'kwoty mleczne': 5,
 'wydają dyrektorzy': 5,
 'terenowych agencji': 5,
 'tożsamość albo': 5,
 'wykonujące badania': 5,
 'do sprawdzania': 5,
 'badań zawartości': 5,
 'tego dostawcy': 5,
 'którą wiąże': 5,
 'mlecznej na': 5,
 'dzierżawcę lub': 5,
 'wejdzie w': 5,
 'przenoszącej własność': 5,
 'się jeżeli': 5,
 'do właściciela': 5,
 'posiadaczem gospodarstwa': 5,
 'umowa dzierżawy': 5,
 'posiadającymi gospodarstwa': 5,
 'samego oddziału': 5,
 'do przyznanej': 5,
 'lub dostawcą': 5,
 'kopię do

In [26]:
bigrams_dictionary = {k: v for k, v in sorted(bigrams_dictionary.items(), key=lambda item: item[1], reverse=True)[:10]}
bigrams_dictionary  


{'mowa w': 28450,
 'których mowa': 13847,
 'o których': 13839,
 'z dnia': 9513,
 'którym mowa': 9165,
 'o którym': 9155,
 'do spraw': 8680,
 'i nr': 8435,
 'dodaje się': 8190,
 'w drodze': 7092}

# 7. Use log likelihood ratio (LLR) to compute the measure for all pairs of words:

In [13]:
def llr_2x2(k11, k12, k21, k22):
    '''Special case of llr with a 2x2 table'''
    return 2 * (denormEntropy([k11 + k12, k21 + k22]) +
                denormEntropy([k11 + k21, k12 + k22]) -
                denormEntropy([k11, k12, k21, k22]))

def denormEntropy(counts):
    '''Computes the entropy of a list of counts scaled by the sum of the counts. If the inputs sum to one, this is just the normal definition of entropy'''
    counts = list(counts)
    total = float(sum(counts))
    # Note tricky way to avoid 0*log(0)
    return -sum([k * math.log(k / total + (k == 0)) for k in counts])

w1_dict, w2_dict, llr_dict = dict(), dict(), dict()

for k in bigrams_dictionary.keys():
    w1, w2 = k.split()
    if w1 not in w1_dict.keys():
        bigrams = set()
    else:
        bigrams = w1_dict[w1]
    bigrams.add(k)
    w1_dict[w1] = bigrams
    if w2 not in w2_dict.keys():
        bigrams = set()
    else:
        bigrams = w2_dict[w2]
    bigrams.add(k)
    w2_dict[w2] = bigrams

for k, v in bigrams_dictionary.items():
    w1, w2 = k.split()
    b1, b2 = w1_dict[w1], w2_dict[w2]
    k11 = v
    k12 = sum(bigrams_dictionary[i] for i in b1) - v
    k21 = sum(bigrams_dictionary[i] for i in b2) - v
    k22 = bigrams_sum - k11 - k12 - k21
    llr_dict[k] = llr_2x2(k11, k12, k21, k22)

llr_dict


{'mowa w': 176478.90209806873,
 'których mowa': 116487.1890897266,
 'o których': 91317.24360996136,
 'z dnia': 51193.825702226255,
 'którym mowa': 74945.41002563864,
 'o którym': 57819.1330856944,
 'do spraw': 62759.87027216959,
 'i nr': 58749.40208535467,
 'dodaje się': 65564.32445676788,
 'w drodze': 42222.39421694481,
 'na podstawie': 52539.76119496202,
 'stosuje się': 48574.833414177,
 'o której': 34790.62867726898,
 'której mowa': 43844.47067235649,
 'w przypadku': 27817.41625480645,
 'od dnia': 34151.996132001455,
 'w zakresie': 27830.244787672185,
 'właściwy do': 32733.16766981181,
 'zastępuje się': 38678.93476830411,
 'w ustawie': 26562.06496036728,
 'minister właściwy': 49954.972916778206,
 'oraz z': 11964.915378040401,
 'w życie': 24036.44329452887,
 'określonych w': 20973.966119877878,
 'w tym': 16468.997376594343,
 'w terminie': 20210.352217419655,
 'w pkt': 13506.926638255478,
 'a także': 39099.12428182573,
 'ustawie z': 24629.77898132155,
 'ustawy z': 13184.566960427444,


# 8. Sort the word pairs according to that measure in the descending order and display top 10 entries:

In [15]:
llr_dict = {k: v for k, v in sorted(llr_dict.items(), key=lambda item: item[1], reverse=True)[:10]}
llr_dict

{'mowa w': 176478.90209806873,
 'których mowa': 116487.1890897266,
 'o których': 91317.24360996136,
 'którym mowa': 74945.41002563864,
 'dodaje się': 65564.32445676788,
 'do spraw': 62759.87027216959,
 'i nr': 58749.40208535467,
 'o którym': 57819.1330856944,
 'na podstawie': 52539.76119496202,
 'z dnia': 51193.825702226255}

# 12. Answer the following questions:
* Why do we have to filter the bigrams, rather than the token sequence?



* Which measure (PMI, PMI with filtering, LLR) works better for the bigrams and which for the trigrams?

W bigramach warto odfiltrować wyrażenia występujące stosunkowo rzadko lub gdy są to pojedyncze przypadki (PMI lub PMI with filtering), ponieważ w tekstach zdarzają się literówki.

* What types of expressions are discovered by the methods.

PMI - wykrywa wyrażenia występująco rzadko. Im mniej ich jest,tym ma wyższą wartość.

LLR - wykrywa wyrażenia charakterystyczne dla danego języka.

* Can you devise a different type of filtering that would yield better results?

Usuwanie wyrazów występujących bardzo często jak na przykład spójniki.
